 # Парсинг данных и анализ. Кассовые сборы фильмов и бюджеты
 
Поработаем с данными о бюджетах фильмов и финансовых показателях с помощью сайта [The Numbers](https://www.the-numbers.com/movie/budgets/all).

In [ ]:
!pip install fake_useragent

In [2]:
# импортируйте нужные библиотеки
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
from tqdm import tqdm
from time import sleep
import matplotlib
%matplotlib inline 

In [3]:
import warnings
warnings.filterwarnings("ignore")

### Парсинг данных

In [4]:
req = requests.get('https://www.the-numbers.com/movie/budgets/all')
print(req)

<Response [403]>


Мы впервые сталкиваемся с тем, что сервер не отдает нам данные по запросу. Но и из этой ситуации есть выход! Библиотека `fake_useragent`.

In [5]:
ua = UserAgent()
headers = {'User-Agent': ua.chrome}

In [6]:
req = requests.get('https://www.the-numbers.com/movie/budgets/all', headers=headers)
print(req)

<Response [200]>


In [7]:
soup = BeautifulSoup(req.text, 'html')

In [11]:
df = pd.read_html(str(soup.find('table')))[0]

In [12]:
df

,Unnamed: 0,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross
0,1,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,"$533,200,000","$936,662,225","$2,064,615,817"
1,2,"Dec 9, 2022",Avatar: The Way of Water,"$460,000,000","$684,075,767","$2,317,514,386"
2,3,"Jun 28, 2023",Indiana Jones and the Dial of Destiny,"$402,300,000","$174,480,468","$383,963,057"
3,4,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,748,242,781"
4,5,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,071,802","$1,045,713,802"
...,...,...,...,...,...,...
95,96,"Nov 22, 2023",Wish,"$200,000,000","$63,973,821","$240,274,313"
96,97,"Feb 28, 2020",Onward,"$200,000,000","$61,555,145","$133,357,601"
97,98,"Nov 22, 2023",Napoleon,"$200,000,000","$61,524,375","$217,917,156"
98,99,"Dec 16, 2020",Wonder Woman 1984,"$200,000,000","$46,801,036","$166,360,232"


Спарсим все данные по ссылке выше.

In [26]:
main = pd.DataFrame()

for i in tqdm(range(1, 66)):
    req = requests.get(f'https://www.the-numbers.com/movie/budgets/all/{i}01', headers=headers)
    soup = BeautifulSoup(req.text, 'html')
    table = soup.find('table')
    df1 = pd.read_html(str(table))[0]
    main = pd.concat([main, df1])
    sleep(3)

100%|███████████████████████████████████████████| 64/64 [03:29<00:00,  3.27s/it]


In [27]:
df = pd.concat([df, main])

In [28]:
df = df.set_index('Unnamed: 0')

In [29]:
df.shape

(6434, 5)

### EDA

* Сколько строк и столбцов содержит набор данных?
* Присутствуют ли значения NaN?
* Есть ли дублирующиеся строки?
* Какие типы данных столбцов?

In [30]:
# ваш код
df.isna().values.any()

False

In [31]:
df.duplicated().values.any()

False

In [ ]:
?df.dropna

In [ ]:
?df.drop_duplicates

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6434 entries, 1 to 6434
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ReleaseDate       6434 non-null   object
 1   Movie             6434 non-null   object
 2   ProductionBudget  6434 non-null   object
 3   DomesticGross     6434 non-null   object
 4   WorldwideGross    6434 non-null   object
dtypes: object(5)
memory usage: 301.6+ KB


### Перевод данных

Переводим столбцы в нужные типы данных

In [33]:
my_cols = list(df.columns)
my_cols

['ReleaseDate', 'Movie', 'ProductionBudget', 'DomesticGross', 'WorldwideGross']

In [ ]:
cols = ['ProductionBudget', 'DomesticGross', 'WorldwideGross']

for col in cols:
    df[col] = df[col].str.replace('$', '')
    df[col] = df[col].str.replace(',', '')
    df[col] = pd.to_numeric(df[col])

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6434 entries, 1 to 6434
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ReleaseDate       6434 non-null   object
 1   Movie             6434 non-null   object
 2   ProductionBudget  6434 non-null   int64 
 3   DomesticGross     6434 non-null   int64 
 4   WorldwideGross    6434 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 301.6+ KB


### Описательная статистика

* Какой средний бюджет фильмов в наборе данных?
* Какой средний мировой доход фильмов?
* Какие минимальные показатели мирового и локального дохода фильмов?
* Какие самый высокий бюджет и самый высокий мировой доход среди фильмов?
* Какой доход принесли фильмы с самым низким и самым высоким бюджетом?

In [51]:
# ваш код
df.describe()

,ProductionBudget,DomesticGross,WorldwideGross
count,"6,434.00","6,434.00","6,434.00"
mean,"32,493,886.84","41,954,185.26","93,284,689.47"
std,"43,755,883.27","71,929,639.07","186,045,067.85"
min,86.00,0.00,0.00
25%,"5,000,000.00","1,084,873.75","3,747,858.75"
50%,"17,000,000.00","16,083,016.50","27,147,670.00"
75%,"40,000,000.00","51,566,931.75","96,671,373.00"
max,"460,000,000.00","936,662,225.00","2,923,706,026.00"


In [ ]:
# ваш код

### Фильмы, потерявшие деньги

* Какой процент фильмов, в которых затраты на производство превысили мировой доход?

In [ ]:
# ваш код